In [1]:
import sys
sys.path.insert(1, '../..')

import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np

from library.evaluation import ConfusionMatrix

random.seed(33)

dataset_name = "Twitter16_CrossVal"
unique_name = "DistilBERT_NLI_Mean"

In [2]:
vectors = np.loadtxt("../../data/processed/vectors/Twitter16_DistilBERT_NLI_Mean_vectors.txt", delimiter=",")
vectors.shape

(818, 768)

In [3]:
data = pd.read_csv("../../data/processed/twitter16_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,tvt,cv_fold,tt
0,656955120626880512,correct predictions in back to the future ii U...,false,training,1,training
1,615689290706595840,.@whitehouse in rainbow colors for #scotusmarr...,true,training,3,training
2,613404935003217920,cops bought the alleged church shooter burger ...,false,training,2,test
3,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified,training,3,test
4,714598641827246081,an open letter to trump voters from his top st...,unverified,training,1,test


In [4]:
labels_str = data['label'].unique().tolist()
labels_str

['false', 'true', 'unverified', 'non-rumor']

In [5]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d['label'])
#     labels.append([1 if j == lab else 0 for j in range(len(labels_str))])
    labels.append(lab)
labels[:10]

[0, 1, 0, 2, 2, 1, 2, 3, 3, 0]

In [6]:
test_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])
test_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])

In [7]:
print(test_vectors.shape)
print(test_labels.shape)

(71, 768)
(71,)


In [8]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import os
from typing import Callable


class NNClassifier(nn.Module):
    def __init__(self,
        n_input: int,
        n_output: int = 1,
        criterion: Callable = nn.BCELoss,
        beta1: float = 0.5,
        lr: float = 0.0002,
        device: str = None
    ):
        super(NNClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(n_input, 512),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(512),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),
#             nn.BatchNorm1d(128),
            nn.Linear(128, n_output),
#             nn.Sigmoid()
        )
        self.criterion = criterion()
        if not device or device not in ['cpu', 'cuda']:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device
        
        self.model = self.model.to(self.device)
        if self.device == 'cuda':
            self.model = torch.nn.DataParallel(self.model)
            cudnn.benchmark = True

        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(beta1, 0.999))

    def forward(self, input):
        return self.model(input)
    
    def load_pretrained(self, filepath: str, key: str = "net", is_parallel: bool = False):
        checkpoint = torch.load(filepath)
        if is_parallel:
            self.model = torch.nn.DataParallel(self.model)
        self.model.load_state_dict(checkpoint[key], strict=False)
    
    def train_eval(self,
        train_x, train_y,
        test_x, test_y,
        n_iter: int = 100,
        batch_size: int = 128,
        saves: str = None
    ):
        trainset = torch.utils.data.TensorDataset(train_x, train_y) # create your datset
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size) # create your dataloader

        testset = torch.utils.data.TensorDataset(test_x, test_y) # create your datset
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size) # create your dataloader

        train_accs = []
        train_losses = []
        test_accs = []
        test_losses = []

        print(f"Using {self.device}")
        best_acc = 0
        best_loss = 1000
        best_test_acc = 0
        epoch = 0
        start_time = time.time()
        results = {}
        while True:
            epoch += 1
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                self.model.zero_grad()
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)

                try:
                    loss = self.criterion(outputs, targets)
                except Exception:
                    loss = self.criterion(outputs, targets.long())

                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                total += targets.size(0)
                
            train_losses.append(train_loss)

            self.model.eval()
            test_loss = 0
            test_acc = 0
            with torch.no_grad():
                inputs, targets = test_x.to(self.device), test_y.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets.long())

                test_loss += loss.item()
                
                preds = self.predict(test_x)
                conf_mat = ConfusionMatrix(
                    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_y]),
                    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds.cpu().numpy()]),
                    binary=False
                )
                conf_mat.evaluate(logs=False)
                test_acc = conf_mat.accuracy

            test_losses.append(test_loss)
            
#             if (epoch) % round(n_iter/20) == 0:
#                 print(f"-- Epoch {epoch}, Train Loss : {train_loss}, Test Loss : {test_loss}")

            # Save checkpoint.
#             if saves and test_loss < best_loss:
#                 print(f"Saving after new best loss : {test_loss}")
#                 best_loss = test_loss
            if saves and test_acc > best_test_acc:
#                 print(f"Saving after new best accuracy : {test_acc}")
                best_test_acc = test_acc

                state = {
                    'net': self.model.state_dict(),
                }
                if not os.path.isdir('models'):
                    os.mkdir('models')
                torch.save(state, f"../../data/models/{saves}.pth")
            
            if epoch >= n_iter:
                break

#         # visualizing accuracy over epoch
#         fig, ax2 = plt.subplots(1)
#         plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=1.5, wspace=0.4)

#         ax2.plot([i for i in range(len(train_losses))], train_losses, c='b', marker="o", label='Train Loss')
#         ax2.plot([i for i in range(len(test_losses))], test_losses, c='r', marker="o", label='Test Loss')
#         ax2.set_ylabel('Loss')
#         ax2.set_xlabel('Epoch')
#         ax2.set_xlim(0, len(train_losses))
#         ax2.set_ylim(min([min(train_losses), min(test_losses)])*0.1, max([max(train_losses), max(test_losses)]))
#         ax2.title.set_text(f"Loss over time (epoch)")
#         ax2.legend(loc='lower right')

#         plt.show()
    
    def predict(self, input_x):
        self.model.eval()
        with torch.no_grad():
            preds = self.model(torch.Tensor(input_x))
            preds = torch.log_softmax(preds, dim = 1)
            _, preds = torch.max(preds, dim = 1)
            return preds

In [9]:
print("3-Fold Cross Validation with 4-Layer Linear Network")

folds = [1, 2, 3]
for fold in folds:
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold}"
    model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)
    model.train_eval(torch.Tensor(train_vectors),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=512)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors)

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with 4-Layer Linear Network
Fold-1 Cross Validation
Using cuda

-------- Fold-1 Results --------
252 vs 252
Multi Class Evaluation

Class false Evaluation
- Precision : 64.384 %
- Recall : 78.333 %
- F1 : 0.70677

Class true Evaluation
- Precision : 90.164 %
- Recall : 91.667 %
- F1 : 0.90909

Class unverified Evaluation
- Precision : 78.431 %
- Recall : 66.667 %
- F1 : 0.72072

Class non-rumor Evaluation
- Precision : 71.642 %
- Recall : 66.667 %
- F1 : 0.69065

Combined Evaluation
- Accuracy : 75.397 %
- Precision : 76.155 %
- Recall : 75.833 %
- F1 : 0.75994

- Average Confidence : 100.0 %
Model, Combined,,,,false,,,true,,,unverified,,,non-rumor,,,
Twitter16_CrossVal_4LayerNet_DistilBERT_NLI_Mean_fold-1, 75.397, 76.155, 75.833, 0.75994, 64.384, 78.333, 0.70677, 90.164, 91.667, 0.90909, 78.431, 66.667, 0.72072, 71.642, 66.667, 0.69065, 
-------- Fold-1 End --------

Fold-2 Cross Validation
Using cuda

-------- Fold-2 Results --------
252 vs 252
Multi Class Eva

In [10]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1

-------- Testing Results --------
71 vs 71
Multi Class Evaluation

Class false Evaluation
- Precision : 61.905 %
- Recall : 81.25 %
- F1 : 0.7027

Class true Evaluation
- Precision : 94.737 %
- Recall : 100.0 %
- F1 : 0.97297

Class unverified Evaluation
- Precision : 77.778 %
- Recall : 73.684 %
- F1 : 0.75676

Class non-rumor Evaluation
- Precision : 84.615 %
- Recall : 61.111 %
- F1 : 0.70968

Combined Evaluation
- Accuracy : 78.873 %
- Precision : 79.759 %
- Recall : 79.011 %
- F1 : 0.79383

- Average Confidence : 100.0 %
Model, Combined,,,,false,,,true,,,unverified,,,non-rumor,,,
Twitter16_CrossVal_4LayerNet_DistilBERT_NLI_Mean_fold-1, 78.873, 79.759, 79.011, 0.79383, 61.905, 81.25, 0.7027, 94.737, 100.0, 0.97297, 77.778, 73.684, 0.75676, 84.615, 61.111, 0.70968, 
-------- Testing End --------



In [11]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2

-------- Testing Results --------
71 vs 71
Multi Class Evaluation

Class false Evaluation
- Precision : 66.667 %
- Recall : 75.0 %
- F1 : 0.70588

Class true Evaluation
- Precision : 80.952 %
- Recall : 94.444 %
- F1 : 0.87179

Class unverified Evaluation
- Precision : 72.222 %
- Recall : 68.421 %
- F1 : 0.7027

Class non-rumor Evaluation
- Precision : 78.571 %
- Recall : 61.111 %
- F1 : 0.6875

Combined Evaluation
- Accuracy : 74.648 %
- Precision : 74.603 %
- Recall : 74.744 %
- F1 : 0.74673

- Average Confidence : 100.0 %
Model, Combined,,,,false,,,true,,,unverified,,,non-rumor,,,
Twitter16_CrossVal_4LayerNet_DistilBERT_NLI_Mean_fold-2, 74.648, 74.603, 74.744, 0.74673, 66.667, 75.0, 0.70588, 80.952, 94.444, 0.87179, 72.222, 68.421, 0.7027, 78.571, 61.111, 0.6875, 
-------- Testing End --------



In [12]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_4LayerNet_{unique_name}_fold-{fold_n}"
model = NNClassifier(train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss)

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors)

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3

-------- Testing Results --------
71 vs 71
Multi Class Evaluation

Class false Evaluation
- Precision : 57.895 %
- Recall : 68.75 %
- F1 : 0.62857

Class true Evaluation
- Precision : 88.889 %
- Recall : 88.889 %
- F1 : 0.88889

Class unverified Evaluation
- Precision : 77.778 %
- Recall : 73.684 %
- F1 : 0.75676

Class non-rumor Evaluation
- Precision : 62.5 %
- Recall : 55.556 %
- F1 : 0.58824

Combined Evaluation
- Accuracy : 71.831 %
- Precision : 71.765 %
- Recall : 71.72 %
- F1 : 0.71742

- Average Confidence : 100.0 %
Model, Combined,,,,false,,,true,,,unverified,,,non-rumor,,,
Twitter16_CrossVal_4LayerNet_DistilBERT_NLI_Mean_fold-3, 71.831, 71.765, 71.72, 0.71742, 57.895, 68.75, 0.62857, 88.889, 88.889, 0.88889, 77.778, 73.684, 0.75676, 62.5, 55.556, 0.58824, 
-------- Testing End --------

